# Embeddings Part 3
# Going Through the Use Cases
Here we show some representative use cases. We will use the Amazon fine-food reviews dataset for the following examples.


In [37]:
# import our packages
import os
import pandas as pd
import numpy as np
from openai import OpenAI

In [38]:
# Create an instance of the OpenAI client
client = OpenAI()

First, let's see what trying to answer without using RAG looks like

In [39]:
from openai import OpenAI

# an example question about the Amazon reviews
query = "What people hated their food?"

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about Amazon fine food reviews.'},
        {'role': 'user', 'content': query},
    ],
    model='gpt-4-turbo',
    temperature=0,
)

print(response.choices[0].message.content)

As an AI, I don't have direct access to specific Amazon fine food reviews or the ability to browse the internet. However, I can provide general insights based on common themes often found in negative food reviews. Typically, people might leave negative reviews due to reasons such as:

1. **Poor Taste or Flavor**: Some reviewers might find the food not to their liking due to blandness, an overpowering taste, or simply because it didn't meet their expectations based on the description.

2. **Quality Issues**: This can include receiving spoiled, stale, or contaminated food. Quality can also refer to the texture or freshness of the product.

3. **Packaging Problems**: Damaged packaging, difficulties in opening the product, or packaging that doesn't preserve the food well can lead to negative reviews.

4. **Price Concerns**: Customers might feel that the product was not worth the price they paid, either due to size, quantity, or overall quality.

5. **Incorrect Product Description**: If the

## Obtaining the Embeddings
The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. We will use a subset of 1,000 most recent reviews for illustration purposes. The reviews are in English and tend to be positive or negative. Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text). For example:

| PRODUCT ID  | USER ID        | SCORE | SUMMARY             | TEXT                                           |
|-------------|----------------|-------|---------------------|------------------------------------------------|
| B001E4KFG0  | A3SGXH7AUHU8GW | 5     | Good Quality Dog Food | I have bought several of the Vitality canned... |
| B00813GRG4  | A1D87F6ZCVE5NK | 1     | Not as Advertised   | Product arrived labeled as Jumbo Salted Peanut... |



We will combine the review summary and review text into a single combined text. The model will encode this combined text and output a single vector embedding.

In [40]:
# utility function to get the file size
def get_file_size(file_path):
    """ Returns the size of the file in megabytes. """
    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)  # Convert from bytes to megabytes
    return size_mb

# main utility function to get the embeddings
def get_embedding(text, model="text-embedding-3-small"):
    # Replace newlines in the text with spaces for consistent formatting
    text = text.replace("\n", " ")
    # Request the embedding for the cleaned text and return the embedding
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# utility function to get the embeddings with reduced dimensions
def get_embedding_reduced_dims(text, model="text-embedding-3-large"):
    # Replace newlines in the text with spaces for consistent formatting
    text = text.replace("\n", " ")
    # Request the embedding for the cleaned text and return the embedding
    return client.embeddings.create(input=[text], model=model,dimensions=1024).data[0].embedding

# utility function to get the embeddings with reduced dimensions
def get_embedding_large(text, model="text-embedding-3-large"):
    # Replace newlines in the text with spaces for consistent formatting
    text = text.replace("\n", " ")
    # Request the embedding for the cleaned text and return the embedding
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [41]:

# Define the path to the input data file
input_data_path = "./EmbeddingsDemoAssets/fine_food_reviews_1k.csv"

# Read the CSV file using pandas and set the first column as the index
df = pd.read_csv(input_data_path, index_col=0)

# Select only the relevant columns from the dataframe
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]

# Drop rows with any missing values
df = df.dropna()

# Drop rows with any missing values
df = df.drop_duplicates()

# Combine the 'Summary' and 'Text' columns into a new column with a formatted string
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)

# Display the first 5 rows of the modified dataframe
df.head(5)


,Time,ProductId,UserId,Score,Summary,Text,combined
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ...."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...


### Small Model Embedding
Now let's embed using the small model first

In [42]:

# Apply the `get_embedding` function to each entry in the 'combined' column and store the results
df['embedding'] = df['combined'].apply(lambda x: get_embedding(x))

# Save the dataframe with embeddings to a CSV file, omitting the index
df.to_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_1k.csv', index=False)

# Display the first 5 rows of the modified dataframe
df.head(5)


,Time,ProductId,UserId,Score,Summary,Text,combined,embedding
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[0.0359942801296711, -0.02117965929210186, -0...."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,"[0.01144526805728674, 0.0342588946223259, -0.0..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....","[0.003300713375210762, 0.012644506990909576, -..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,"[-0.002887785667553544, 0.014631008729338646, ..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,"[0.012054475955665112, -0.05606513097882271, 0..."


In [43]:
# Get the size of the data file
file_path = './EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_1k.csv'
size_mb = get_file_size(file_path)
print(f"The size of the file is {size_mb:.2f} MB.")

The size of the file is 33.43 MB.


### Large Model Embedding
Let's embed using the large model next.

In [44]:

# Apply the `get_embedding_large` function to each entry in the 'combined' column and store the results
df['embedding'] = df['combined'].apply(lambda x: get_embedding_large(x))

# Save the dataframe with embeddings to a CSV file, omitting the index
df.to_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv', index=False)

# Display the first 5 rows of the modified dataframe
df.head(5)

,Time,ProductId,UserId,Score,Summary,Text,combined,embedding
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[-0.01042067352682352, 0.006103187799453735, -..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,"[0.007289395667612553, -0.026579800993204117, ..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....","[-0.003412495134398341, 0.005741763859987259, ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,"[0.011873782612383366, -0.012579566799104214, ..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,"[-0.011918674223124981, 0.01122630387544632, 0..."


In [45]:
# Get the size of the data file
file_path = './EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv'
size_mb = get_file_size(file_path)
print(f"The size of the file is {size_mb:.2f} MB.")

The size of the file is 66.51 MB.


### Loading the Data
Here we show how to load the data into a dataframe from a CSV file to make it ready to be used again when needed. 

In [46]:

# Load the CSV file into a pandas DataFrame
df_load = pd.read_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv')

# Convert the string representation of embeddings in the 'embedding' column to numpy arrays
# By converting the embeddings from string format to numpy arrays immediately after loading, 
# we ensure that the data is in a ready-to-use state for any subsequent analysis or processing steps.
df_load['embedding'] = df_load['embedding'].apply(eval).apply(np.array)

# Display the first 5 rows of the loaded dataframe
df_load.head(5)

,Time,ProductId,UserId,Score,Summary,Text,combined,embedding
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[-0.01042067352682352, 0.006103187799453735, -..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,"[0.007289395667612553, -0.026579800993204117, ..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....","[-0.003412495134398341, 0.005741763859987259, ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,"[0.011873782612383366, -0.012579566799104214, ..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,"[-0.011918674223124981, 0.01122630387544632, 0..."


### Reducing the Dimensions
Using larger embeddings, for example storing them in a vector store for retrieval, generally costs more and consumes more compute, memory and storage than using smaller embeddings.

Both of our new embedding models were trained with a technique that allows developers to trade-off performance and cost of using embeddings. Specifically, developers can shorten embeddings (i.e. remove some numbers from the end of the sequence) without the embedding losing its concept-representing properties.

In [47]:

# Define the path to the input data file
input_data_path = "./EmbeddingsDemoAssets/fine_food_reviews_1k.csv"

# Read the CSV file using pandas and set the first column as the index
df_reduced_dims = pd.read_csv(input_data_path, index_col=0)

# Select only the relevant columns from the dataframe
df_reduced_dims = df_reduced_dims[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]

# Drop rows with any missing values
df_reduced_dims = df_reduced_dims.dropna()

# Drop rows with any missing values
df_reduced_dims = df_reduced_dims.drop_duplicates()

# Combine the 'Summary' and 'Text' columns into a new column with a formatted string
df_reduced_dims["combined"] = (
    "Title: " + df_reduced_dims.Summary.str.strip() + "; Content: " + df_reduced_dims.Text.str.strip()
)

# Display the first 5 rows of the modified dataframe
df_reduced_dims.head(5)

,Time,ProductId,UserId,Score,Summary,Text,combined
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ...."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...


In [48]:

# Apply the `get_embedding` function to each entry in the 'combined' column and store the results
df_reduced_dims['embedding'] = df_reduced_dims['combined'].apply(lambda x: get_embedding_reduced_dims(x))

# Save the dataframe with embeddings to a CSV file, omitting the index
df_reduced_dims.to_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_reduced_dims_1k.csv', index=False)

# Display the first 5 rows of the modified dataframe
df.head(5)


,Time,ProductId,UserId,Score,Summary,Text,combined,embedding
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[-0.01042067352682352, 0.006103187799453735, -..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,"[0.007289395667612553, -0.026579800993204117, ..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....","[-0.003412495134398341, 0.005741763859987259, ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,"[0.011873782612383366, -0.012579566799104214, ..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,"[-0.011918674223124981, 0.01122630387544632, 0..."


In [49]:

# Get the size of the data file
file_path = './EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_reduced_dims_1k.csv'
size_mb = get_file_size(file_path)
print(f"The size of the file is {size_mb:.2f} MB.")

The size of the file is 22.46 MB.


## Question Answering Using Embeddings-Based Search
There are many common cases where the model is not trained on data which contains key facts and information you want to make accessible when generating responses to a user query. One way of solving this is to put additional information into the context window of the model. This is effective in many use cases but leads to higher token costs. The other way is to use RAG to obtain the information. 

In [50]:

# Load the CSV file into a pandas DataFrame
df_search = pd.read_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv')

# Convert the string representation of embeddings in the 'embedding' column to numpy arrays
# By converting the embeddings from string format to numpy arrays immediately after loading, 
# we ensure that the data is in a ready-to-use state for any subsequent analysis or processing steps.
df_search['embedding'] = df_search['embedding'].apply(eval).apply(np.array)

# Display the first 5 rows of the loaded dataframe
df_search.head(5)

,Time,ProductId,UserId,Score,Summary,Text,combined,embedding
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[-0.01042067352682352, 0.006103187799453735, -..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,"[0.007289395667612553, -0.026579800993204117, ..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....","[-0.003412495134398341, 0.005741763859987259, ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,"[0.011873782612383366, -0.012579566799104214, ..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,"[-0.011918674223124981, 0.01122630387544632, 0..."


In [53]:
# Import necessary libraries
from scipy.spatial import distance
import pandas as pd

# Define a function that ranks strings from a pandas DataFrame based on their relatedness to a given query string.
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - distance.cosine(x, y),
    top_n: int = 100,
    threshold: float = 0.001  # Minimum score difference to consider for ranking
) -> tuple[list[str], list[float]]:
    """
    Retrieve the top 'n' strings related to a query string from a DataFrame, based on a custom relatedness function.

    Parameters:
    query (str): The string to compare other strings against.
    df (pd.DataFrame): DataFrame containing the strings and their embeddings.
    relatedness_fn (callable): A function that computes the relatedness score between two embeddings. By default, 
                               it uses the cosine similarity between embeddings.
    top_n (int): The number of top related strings to return.
    threshold (float): The minimum difference between scores needed to consider one string more related than another.

    Returns:
    tuple[list[str], list[float]]: A tuple containing two lists:
                                   1. The top 'n' strings most related to the query.
                                   2. Their corresponding relatedness scores.
    """
    
    # Retrieve the embedding for the query string using a pre-defined model.
    query_embedding_response = client.embeddings.create(
        model="text-embedding-3-large",
        input=query,
    )
    # Extract the embedding data from the response.
    query_embedding = query_embedding_response.data[0].embedding

    # Compute the relatedness of each string in the DataFrame to the query string.
    strings_and_relatednesses = [
        (row["combined"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]

    # Sort the list of tuples by the relatedness score in descending order (most related first).
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Initialize a list to store the filtered results and a variable to track the last accepted score.
    filtered = []
    last_score = -1

    # Filter strings to meet the threshold criteria and limit the number of results to 'top_n'.
    for item in strings_and_relatednesses:
        if abs(item[1] - last_score) > threshold:
            filtered.append(item)
            last_score = item[1]
        if len(filtered) >= top_n:
            break

    # Unzip the tuples to separate strings and their relatedness scores.
    strings, relatednesses = zip(*filtered)

    # Return only the top 'n' results as specified by the function arguments.
    return strings[:top_n], relatednesses[:top_n]



In [57]:
# This code performs a search to find the top 5 items related to the phrase "don't like the taste" 
# from a DataFrame 'df_search' using the previously defined 'strings_ranked_by_relatedness' function.

# Call the function 'strings_ranked_by_relatedness' with the query string, DataFrame, and specify the number 
# of top related items to return (top_n=5).
strings, relatednesses = strings_ranked_by_relatedness("don't like the taste", df_search, top_n=5)

# Loop over each string and its corresponding relatedness score.
# The 'zip' function combines the two lists 'strings' and 'relatednesses' so that items from both lists 
# can be accessed in a single loop iteration.
for string, relatedness in zip(strings, relatednesses):
    # Print a formatted string that includes a visual separator and the relatedness score formatted to three decimal places.
    print(f"\n========================\n{relatedness=:.3f}")
    
    # The 'display' function is typically used in Jupyter Notebooks or similar environments to render objects in a more
    # visually appealing manner than the basic print function. Here, it is used to display the string from the DataFrame.
    display(string)



relatedness=0.658


"Title: Don't like the taste; Content: I do not like sour taste and this has a sour kind of taste which i don't like. The smell isn't that great either"


relatedness=0.493


"Title: Doesn't taste good...; Content: I didn't like the flavor of this root beer snow cone syrup, it has a bitter flavor.<br /><br />It is a great deal tho so it's too bad it doesn't taste good. :("


relatedness=0.472


"Title: No Thanks!; Content: I LOVE the Blue Sky Wild Raspberry but I cannot stand the Black Cherry.  It doesn't have much flavor and it has an awful aftertaste.  UGH!"


relatedness=0.452


"Title: Didn't like it; Content: Quite personally, I didn't like it.  For me, it had no flavour at all.  It only tasted better when I added some nutmeg (ran out of cinnamon) and milk.  On the brightside, it did help me get rid of my cold :')"


relatedness=0.445


'Title: I did not like it.; Content: I did not like the taste of this Illy cappuccino drink.  I like coffee and coffee drinks in general, and I was looking forward to trying something new, but this product had a slight metallic/chemical aftertaste that, while not strong, was present in sufficient strength to make it less than pleasing to me.  I much prefer the Starbucks Mocha Cappuccino drink that tastes only of coffee, milk, sugar and chocolate.  I do not know if it is the metal container that imparts the aftertaste to the Illy product.  The similar Starbucks drink comes in a glass container that I suppose could have something to do with its fresher taste.  Also, since my opinion is certainly subjective, others may like the Illy drink and be just as justified in their opinion.'